# Policy Gradient Theorem

$$\nabla_{\theta}J(\theta) = \int_S \rho^{\pi}(s) \int_A \nabla_{\theta}\pi(s,a;\theta) Q^\pi (s,a)\operatorname{d}\!a\operatorname{d}\!s$$

# Proof

$$J(\theta) 
= \int_S p_0(s_0) V^\pi(s_0) \operatorname{d}\!s_0 
= \int_S p_0(s_0) \int_{A} \pi(s_0, a_0 ; \theta) Q^\pi(s_0,a_0) \operatorname{d}\!a_0 \operatorname{d}\!s_0$$

Hence,

$$\nabla_{\theta} J(\theta) 
= \int_S p_0(s_0) \int_{A} \nabla_{\theta} \pi(s_0, a_0 ; \theta) Q^\pi(s_0,a_0) \operatorname{d}\!a_0 \operatorname{d}\!s_0
+ \int_S p_0(s_0) \int_{A} \pi(s_0, a_0 ; \theta) \nabla_{\theta} Q^\pi(s_0,a_0) \operatorname{d}\!a_0 \operatorname{d}\!s_0$$

The second term in the sum is equal to 

\begin{align*}
\int_S p_0(s_0) \int_{A} \pi(s_0, a_0 ; \theta) \nabla_{\theta} Q^\pi(s_0,a_0) \operatorname{d}\!a_0 \operatorname{d}\!s_0
&= \int_S p_0(s_0) \int_A \pi(s_0, a_0 ; \theta) \nabla_{\theta} \left(\int_S \gamma \mathcal P_{s0,s1}^{a_0}V^\pi(s_1) \operatorname{d}\!s_1\right) \operatorname{d}\! a_0 \operatorname{d}\! s_0 \\
&= \int_S \left( \int_S \gamma p_0(s_0) \int_A \pi(s_0, a_0 ; \theta)P_{s_0,s_1}^{a_0}   \operatorname{d}\! a_0 \operatorname{d}\! s_0 \right) \nabla_{\theta}V^\pi(s_1) \operatorname{d}\! s_1 \\
&= \int_S \left( \int_S \gamma p_0(s_0) p(s_0 \rightarrow s_1, 1, \pi) \operatorname{d}\! s_0 \right) \nabla_{\theta} \left(\int_A \pi(s_1, a_1 ; \theta) Q^\pi(s_1, a_1) \operatorname{d}\! a_1 \right) \operatorname{d}\! s_1 \\
\end{align*}

Iterating yields
$$\sum_{t=0}^\infty \int_S \int_S \gamma^t p_0(s_0) p(s_0 \rightarrow s_t, t, \pi) \operatorname{d}\! s_0 \int_A \nabla_\theta \pi(s_t, a_t; \theta) Q^\pi(s_t, a_t) \operatorname{d}\! a_t \operatorname{d}\! s_t$$

Interverting the sum and the integral gives
$$\int_S \int_S \sum_{t=0}^\infty \gamma^t p_0(s_0) p(s_0 \rightarrow s_t, t, \pi) \operatorname{d}\! s_0 \int_{A} \nabla_{\theta}\pi(s,a;\theta) Q^\pi (s,a)\operatorname{d}\!a\operatorname{d}\!s$$

i.e. 
$$\int_S \rho^\pi(s) \int_{A} \nabla_{\theta}\pi(s,a;\theta) Q^\pi (s,a)\operatorname{d}\!a\operatorname{d}\!s$$

# Softmax Scores

$\forall s \in S, a \in A, \, \pi(s, a; \theta) = \frac{e^{\theta^{T} \phi(s, a)}}{\sum_{b} e^{\theta^{T} \phi(s, b)}}$

The score function is $$\nabla_{\theta} \log \pi(s, a; \theta) = \phi(s, a) - \sum_{b} \pi(s, b  ; \theta) \phi(s, b) = \phi(s, a) - \mathbb{E}_{\pi}[\phi(s, \cdot)]$$

# Gaussian Scores

The score function is $$\nabla_{\theta} \log \pi(s, a; \theta) = \frac{(a-\theta^T\phi(s))\phi(s)}{\sigma^2}$$

# REINFORCE

In [1]:
from lib.reinforce import *

# Compatible Function Approximation Theorem

If 
- the Critic gradient is compatible with the Actor score function 
$$\nabla_w Q(s, a; w) = \nabla_\theta \log\pi(s, a; \theta)$$
- the Critic parameters $w$ minimize the following mean-squared error
$$\varepsilon = \int_S \rho^\pi(s) \int_A \pi(s, a; \theta) \left(Q^\pi(s,a) - Q(s,a; w)\right)^2 \operatorname{d}\!a \operatorname{d}\! s$$

Then the Policy Gradient using critic $Q(s, a; w)$ is exact
$$\nabla_\theta J(\theta) = \int_S \rho^\pi(s) \int_A \nabla_\theta  \pi(s, a; \theta) Q(s,a; w)\operatorname{d}\!a \operatorname{d}\! s$$

# Proof

For $w$ that minimizes $\varepsilon$, we have 
\begin{align*}
0 &= \int_S \rho^\pi(s) \int_A \pi(s, a; \theta) \left(Q^\pi(s,a) - Q(s,a; w)\right) \nabla_w Q(s,a; w) \operatorname{d}\!a \operatorname{d}\! s \\
&= \int_S \rho^\pi(s) \int_A \pi(s, a; \theta) \left(Q^\pi(s,a) - Q(s,a; w)\right) \nabla_\theta \log\pi(s, a; \theta) \operatorname{d}\!a \operatorname{d}\! s 
\end{align*}
Hence, $$\int_S \rho^\pi(s) \int_A \pi(s, a; \theta) Q^\pi(s,a)\nabla_\theta \log\pi(s, a; \theta) \operatorname{d}\!a \operatorname{d}\! s = \int_S \rho^\pi(s) \int_A \pi(s, a; \theta) Q(s,a; w) \nabla_\theta \log\pi(s, a; \theta) \operatorname{d}\!a \operatorname{d}\! s $$
i.e.
\begin{align*}
\nabla_\theta J(\theta) &= \int_S \rho^\pi(s) \int_A \pi(s, a; \theta) Q(s,a; w) \nabla_\theta \log\pi(s, a; \theta) \operatorname{d}\!a \operatorname{d}\! s\\
&= \int_S \rho^\pi(s) \int_A \nabla_\theta  \pi(s, a; \theta) Q(s,a; w)\operatorname{d}\!a \operatorname{d}\! s
\end{align*}